# Compare unassembled MAC and MIC libraries by k-mers

For k-mer comparison we want to downsample reads to have similar coverage, and to compare R1 vs R2 reads as a sanity check, as [recommended in Kat docs](https://kat.readthedocs.io/en/latest/walkthrough.html#comparing-r1-v-r2-in-an-illumina-pe-dataset).

Reads per library: Lmag MAC and MIC, two different sorting runs sequenced as separate libraries per nucleus type, with additional resequencing per library.

Target | FACS experiments | Library | Reads    | Reseq  | Reseq reads
-------|------------------|---------|----------|--------|-------------
MAC    | 273-5U, 273-6U   | 4519.A  | 45451157 | 4615.A | 95684586
MIC    | 273-5L, 273-6L   | 4519.B  | 47847972 | 4615.B | 93393507
MAC    | 279-2U           | 4519.C  | 34013629 | 4615.C | 93084490
MIC    | 279-2L           | 4519.D  | 44700862 | 4615.D | 94529508

Final coverage is about 100x for ~280M reads pooled per library. Downsample of 90M reads per sample would therefore represent about 30x coverage.

In [18]:
%%bash
source activate bbmap
for i in A B C D; do
    reformat.sh threads=8 \
      in=/ebio/abt2_projects/ag-swart-loxodes/data/reads-trim/4615_${i}_R1_ktrim_qtrim28.fq.gz \
      in2=/ebio/abt2_projects/ag-swart-loxodes/data/reads-trim/4615_${i}_R2_ktrim_qtrim28.fq.gz \
      samplereadstarget=150000000 \
      out=reads_downsampled/4615_${i}_R1_ktrim_qtrim28.150M.fq.gz \
      out2=reads_downsampled/4615_${i}_R2_ktrim_qtrim28.150M.fq.gz
done

java -ea -Xmx200m -cp /ebio/ag-swart/home/kbseah/anaconda3/envs/bbmap/opt/bbmap-38.22-0/current/ jgi.ReformatReads threads=8 in=/ebio/abt2_projects/ag-swart-loxodes/data/reads-trim/4615_A_R1_ktrim_qtrim28.fq.gz in2=/ebio/abt2_projects/ag-swart-loxodes/data/reads-trim/4615_A_R2_ktrim_qtrim28.fq.gz samplereadstarget=150000000 out=reads_downsampled/4615_A_R1_ktrim_qtrim28.150M.fq.gz out2=reads_downsampled/4615_A_R2_ktrim_qtrim28.150M.fq.gz
Executing jgi.ReformatReads [threads=8, in=/ebio/abt2_projects/ag-swart-loxodes/data/reads-trim/4615_A_R1_ktrim_qtrim28.fq.gz, in2=/ebio/abt2_projects/ag-swart-loxodes/data/reads-trim/4615_A_R2_ktrim_qtrim28.fq.gz, samplereadstarget=150000000, out=reads_downsampled/4615_A_R1_ktrim_qtrim28.150M.fq.gz, out2=reads_downsampled/4615_A_R2_ktrim_qtrim28.150M.fq.gz]

Set threads to 8
Set INTERLEAVED to false
Input is being processed as paired
Input:                  	344084052 reads          	41313938496 bases
Output:                 	300000000 reads (87.19%) 	

In [ ]:
%%bash
# COMPARE R1 R2 OF SAME LIBRARY
set -e
source activate /ebio/abt2_projects/ag-swart-loxodes/envs/kat

for i in B C D; do
    kat comp -t 16 -m 19 -n -o kat-comp_k-19_LmagMAC_4615_${i}_150M_R1_v_R2 \
     reads_downsampled/4615_${i}_R1_ktrim_qtrim28.150M.fq.gz \
     reads_downsampled/4615_${i}_R2_ktrim_qtrim28.150M.fq.gz &> kat-comp_k-19_LmagMAC_4615_${i}_150M_R1_v_R2.log
done

. | A  | B  | C  | D |
--|----|----|----|---|
A | -  | x  | o  | x |
B |    | -  | x  | o | 
C |    |    | -  | x |
D |    |    |    | - |

In [39]:
%%bash
# COMPARE DIFFERENT LIBRARIES FROM MAC vs MAC
set -e
source activate /ebio/abt2_projects/ag-swart-loxodes/envs/kat
kat comp -t 16 -m 19 -n -o kat-comp_k-19_LmagMAC-LmagMAC_4615_A_v_C_150M \
 'reads_downsampled/4615_A_R?_ktrim_qtrim28.150M.fq.gz' \
 'reads_downsampled/4615_C_R?_ktrim_qtrim28.150M.fq.gz' &> kat-comp_k-19_LmagMAC-LmagMAC_4615_A_v_C_150M.log

In [40]:
%%bash
# COMPARE DIFFERENT LIBRARIES FROM MIC vs MIC
set -e
source activate /ebio/abt2_projects/ag-swart-loxodes/envs/kat
kat comp -t 16 -m 19 -n -o kat-comp_k-19_LmagMAC-LmagMAC_4615_B_v_D_150M \
 'reads_downsampled/4615_B_R?_ktrim_qtrim28.150M.fq.gz' \
 'reads_downsampled/4615_D_R?_ktrim_qtrim28.150M.fq.gz' &> kat-comp_k-19_LmagMIC-LmagMIC_4615_B_v_D_150M.log

In [41]:
%%bash
# CROSS COMPARE MIC vs MAC
set -e
source activate /ebio/abt2_projects/ag-swart-loxodes/envs/kat
kat comp -t 16 -m 19 -n -o kat-comp_k-19_LmagMAC-LmagMAC_4615_A_v_B_150M \
 'reads_downsampled/4615_A_R?_ktrim_qtrim28.150M.fq.gz' \
 'reads_downsampled/4615_B_R?_ktrim_qtrim28.150M.fq.gz' &> kat-comp_k-19_LmagMAC-LmagMIC_4615_A_v_B_150M.log

In [42]:
%%bash
# CROSS COMPARE MIC vs MAC
set -e
source activate /ebio/abt2_projects/ag-swart-loxodes/envs/kat
kat comp -t 16 -m 19 -n -o kat-comp_k-19_LmagMAC-LmagMAC_4615_C_v_D_150M \
 'reads_downsampled/4615_C_R?_ktrim_qtrim28.150M.fq.gz' \
 'reads_downsampled/4615_D_R?_ktrim_qtrim28.150M.fq.gz' &> kat-comp_k-19_LmagMAC-LmagMIC_4615_C_v_D_150M.log

In [43]:
%%bash
# CROSS COMPARE MIC vs MAC
set -e
source activate /ebio/abt2_projects/ag-swart-loxodes/envs/kat
kat comp -t 16 -m 19 -n -o kat-comp_k-19_LmagMAC-LmagMAC_4615_A_v_D_150M \
 'reads_downsampled/4615_A_R?_ktrim_qtrim28.150M.fq.gz' \
 'reads_downsampled/4615_D_R?_ktrim_qtrim28.150M.fq.gz' &> kat-comp_k-19_LmagMAC-LmagMIC_4615_A_v_D_150M.log

In [44]:
%%bash
# CROSS COMPARE MIC vs MAC
set -e
source activate /ebio/abt2_projects/ag-swart-loxodes/envs/kat
kat comp -t 16 -m 19 -n -o kat-comp_k-19_LmagMAC-LmagMAC_4615_B_v_C_150M \
 'reads_downsampled/4615_B_R?_ktrim_qtrim28.150M.fq.gz' \
 'reads_downsampled/4615_C_R?_ktrim_qtrim28.150M.fq.gz' &> kat-comp_k-19_LmagMAC-LmagMIC_4615_B_v_C_150M.log

In [ ]:
%%bash
# ASSEMBLY SPECTRUM Illumina downsampled vs MAC-assem
set -e
source activate /ebio/abt2_projects/ag-swart-loxodes/envs/kat
REF=/ebio/abt2_projects/ag-swart-loxodes/assembly/flye-comb_LmagMAC/assembly.fasta

kat comp -t 16 -m 19 \
  -o kat-comp_k-19_LmagMAC_4615_A_150M_v_flye-comb_LmagMAC \
  'reads_downsampled/4615_A_R?_ktrim_qtrim28.150M.fq.gz' \
  $REF \
  &> kat-comp_k-19_LmagMAC_4615_A_150M_v_flye-comb_LmagMAC.log

kat comp -t 16 -m 19 \
  -o kat-comp_k-19_LmagMAC_4615_B_150M_v_flye-comb_LmagMAC \
  'reads_downsampled/4615_B_R?_ktrim_qtrim28.150M.fq.gz' \
  $REF \
  &> kat-comp_k-19_LmagMAC_4615_B_150M_v_flye-comb_LmagMAC.log

kat comp -t 16 -m 19 \
  -o kat-comp_k-19_LmagMAC_4615_C_150M_v_flye-comb_LmagMAC \
  'reads_downsampled/4615_C_R?_ktrim_qtrim28.150M.fq.gz' \
  $REF \
  &> kat-comp_k-19_LmagMAC_4615_C_150M_v_flye-comb_LmagMAC.log

kat comp -t 16 -m 19 \
  -o kat-comp_k-19_LmagMAC_4615_D_150M_v_flye-comb_LmagMAC \
  'reads_downsampled/4615_D_R?_ktrim_qtrim28.150M.fq.gz' \
  $REF \
  &> kat-comp_k-19_LmagMAC_4615_D_150M_v_flye-comb_LmagMAC.log

In [58]:
%%bash

for i in *R2.stats; do
  echo $i
  grep -A5 'Distance' $i
  echo ""
done

kat-comp_k-19_LmagMAC_4615_A_150M_R1_v_R2.stats
Distance between spectra 1 and 2 (all k-mers):
 - Manhattan distance: 7.03563e+07
 - Euclidean distance: 1.19481e+07
 - Cosine distance: 0.0302026
 - Canberra distance: 154.746
 - Jaccard distance: 0.256772
--
Distance between spectra 1 and 2 (shared k-mers):
 - Manhattan distance: 6.47774e+07
 - Euclidean distance: 1.04633e+07
 - Cosine distance: 0.0477699
 - Canberra distance: 155.121
 - Jaccard distance: 0.270385

kat-comp_k-19_LmagMAC_4615_B_150M_R1_v_R2.stats
Distance between spectra 1 and 2 (all k-mers):
 - Manhattan distance: 7.81055e+07
 - Euclidean distance: 1.3431e+07
 - Cosine distance: 0.0436127
 - Canberra distance: 201.168
 - Jaccard distance: 0.290165
--
Distance between spectra 1 and 2 (shared k-mers):
 - Manhattan distance: 7.58525e+07
 - Euclidean distance: 1.2879e+07
 - Cosine distance: 0.0650533
 - Canberra distance: 201.756
 - Jaccard distance: 0.311907

kat-comp_k-19_LmagMAC_4615_C_150M_R1_v_R2.stats
Distance between

In [61]:
%%bash

for i in *LmagMAC.stats; do
  echo $i
  grep -A5 'Distance' $i
  echo ""
done

kat-comp_k-19_LmagMAC_4615_A_150M_v_flye-comb_LmagMAC.stats
Distance between spectra 1 and 2 (all k-mers):
 - Manhattan distance: 3.58564e+08
 - Euclidean distance: 1.32904e+08
 - Cosine distance: 0.638038
 - Canberra distance: 978.458
 - Jaccard distance: 0.876324
--
Distance between spectra 1 and 2 (shared k-mers):
 - Manhattan distance: 3.63313e+08
 - Euclidean distance: 1.4367e+08
 - Cosine distance: 0.994682
 - Canberra distance: 980.492
 - Jaccard distance: 0.981311

kat-comp_k-19_LmagMAC_4615_B_150M_v_flye-comb_LmagMAC.stats
Distance between spectra 1 and 2 (all k-mers):
 - Manhattan distance: 3.53088e+08
 - Euclidean distance: 1.36154e+08
 - Cosine distance: 0.703092
 - Canberra distance: 977.254
 - Jaccard distance: 0.88562
--
Distance between spectra 1 and 2 (shared k-mers):
 - Manhattan distance: 3.61002e+08
 - Euclidean distance: 1.43774e+08
 - Cosine distance: 0.993047
 - Canberra distance: 978.689
 - Jaccard distance: 0.978163

kat-comp_k-19_LmagMAC_4615_C_150M_v_flye-com

In [72]:
%%bash
# ASSEMBLY SPECTRUM Illumina complete vs MAC-assem
set -e
source activate /ebio/abt2_projects/ag-swart-loxodes/envs/kat
REF=/ebio/abt2_projects/ag-swart-loxodes/assembly/flye-comb_LmagMAC/assembly.fasta

kat comp -t 16 -m 19 \
  -o kat-comp_k-19_LmagMAC-comb_v_flye-comb_LmagMAC \
  '/ebio/abt2_projects/ag-swart-loxodes/data/reads-trim/4519_A_R?_ktrim_qtrim28.fq.gz /ebio/abt2_projects/ag-swart-loxodes/data/reads-trim/4519_C_R?_ktrim_qtrim28.fq.gz /ebio/abt2_projects/ag-swart-loxodes/data/reads-trim/4615_A_R?_ktrim_qtrim28.fq.gz /ebio/abt2_projects/ag-swart-loxodes/data/reads-trim/4615_C_R?_ktrim_qtrim28.fq.gz' \
  $REF \
  &> kat-comp_k-19_LmagMAC-comb_v_flye-comb_LmagMAC.log


kat comp -t 16 -m 19 \
  -o kat-comp_k-19_LmagMIC-comb_v_flye-comb_LmagMAC \
  '/ebio/abt2_projects/ag-swart-loxodes/data/reads-trim/4519_B_R?_ktrim_qtrim28.fq.gz /ebio/abt2_projects/ag-swart-loxodes/data/reads-trim/4519_D_R?_ktrim_qtrim28.fq.gz /ebio/abt2_projects/ag-swart-loxodes/data/reads-trim/4615_B_R?_ktrim_qtrim28.fq.gz /ebio/abt2_projects/ag-swart-loxodes/data/reads-trim/4615_D_R?_ktrim_qtrim28.fq.gz' \
  $REF \
  &> kat-comp_k-19_LmagMIC-comb_v_flye-comb_LmagMAC.log